In [17]:
# %matplotlib widget

from __future__ import annotations

import re
from collections import defaultdict
from dataclasses import dataclass, field
from itertools import permutations, product
from math import inf
from random import choice

import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import numpy.typing as npt
from mpl_toolkits.mplot3d import axes3d
from numpy import int_, object_
from numpy.typing import NDArray
from test_utilities import run_tests_params
from util import print_hex

COLORS = list(mcolors.CSS4_COLORS.keys())

<link href="style.css" rel="stylesheet"></link><main>

<article class="day-desc"><h2>--- Day 23: Coprocessor Conflagration ---</h2><p>You decide to head directly to the CPU and fix the printer from there. As you get close, you find an <em>experimental coprocessor</em> doing so much work that the local programs are afraid it will <a href="https://en.wikipedia.org/wiki/Halt_and_Catch_Fire">halt and catch fire</a>. This would cause serious issues for the rest of the computer, so you head in and see what you can do.</p>
<p>The code it's running seems to be a variant of the kind you saw recently on that <a href="18">tablet</a>. The general functionality seems <em>very similar</em>, but some of the instructions are different:</p>
<ul>
<li><code>set X Y</code> <em>sets</em> register <code>X</code> to the value of <code>Y</code>.</li>
<li><code>sub X Y</code> <em>decreases</em> register <code>X</code> by the value of <code>Y</code>.</li>
<li><code>mul X Y</code> sets register <code>X</code> to the result of <em>multiplying</em> the value contained in register <code>X</code> by the value of <code>Y</code>.</li>
<li><code>jnz X Y</code> <em>jumps</em> with an offset of the value of <code>Y</code>, but only if the value of <code>X</code> is <em>not zero</em>. (An offset of <code>2</code> skips the next instruction, an offset of <code>-1</code> jumps to the previous instruction, and so on.)</li>
<p>Only the instructions listed above are used. The eight registers here, named <code>a</code> through <code>h</code>, all start at <code>0</code>.</p>
</ul>
<p>The coprocessor is currently set to some kind of <em>debug mode</em>, which allows for testing, but prevents it from doing any meaningful work.</p>
<p>If you run the program (your puzzle input), <em>how many times is the <code>mul</code> instruction invoked?</em></p>
</article>


In [18]:
from re import compile, match


def coprosser(program: str) -> int:
    registers = defaultdict(int)

    regex = compile(r"^[\-+]?\d+$")

    instuctions = [
        [int(c) if match(regex, c) else c for c in line.split(" ")]
        for line in program.strip().splitlines()
    ]

    counter, count_mul = 0, 0
    while 0 <= counter < len(instuctions):
        match instuctions[counter]:
            case ["set", str(x), str(y)]:
                registers[x] = registers[y]
            case ["set", str(x), int(y)]:
                registers[x] = y
            case ["add", str(x), str(y)]:
                registers[x] += registers[y]
            case ["add", str(x), int(y)]:
                registers[x] += y
            case ["mul", str(x), str(y)]:
                count_mul += 1
                registers[x] *= registers[y]
            case ["mul", str(x), int(y)]:
                count_mul += 1
                registers[x] *= y
            case ["mod", str(x), str(y)]:
                registers[x] %= registers[y]
            case ["mod", str(x), int(y)]:
                registers[x] %= y
            case ["jnz", str(x), str(y)]:
                if registers[x] != 0:
                    counter += registers[y] - 1
            case ["jnz", str(x), int(y)]:
                if registers[x] != 0:
                    counter += y - 1
            case ["jnz", int(x), str(y)]:
                if x != 0:
                    counter += registers[y] - 1
            case ["jnz", int(x), int(y)]:
                if x != 0:
                    counter += y - 1
            case ["sub", str(x), str(y)]:
                registers[x] -= registers[y]
            case ["sub", str(x), int(y)]:
                registers[x] -= y
            case _:
                raise ValueError(f"Unknown instruction {instuctions[counter]}.")

        counter += 1

    return count_mul


with open("../input/day23.txt") as f:
    puzzle = f.read()

print(f"Part I: {coprosser(puzzle)}")

Part I: 5929


<link href="style.css" rel="stylesheet"></link><main>
<main>

<p>Your puzzle answer was <code>5929</code>.</p><p class="day-success">The first half of this puzzle is complete! It provides one gold star: *</p>
<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>Now, it's time to fix the problem.</p>
<p>The <em>debug mode switch</em> is wired directly to register <code>a</code>.  You <span title="From 'magic' to 'more magic'.">flip the switch</span>, which makes <em>register <code>a</code> now start at <code>1</code></em> when the program is executed.</p>
<p>Immediately, the coprocessor begins to overheat.  Whoever wrote this program obviously didn't choose a very efficient implementation.  You'll need to <em>optimize the program</em> if it has any hope of completing before Santa needs that printer working.</p>
<p>The coprocessor's ultimate goal is to determine the final value left in register <code>h</code> once the program completes. Technically, if it had that... it wouldn't even need to run the program.</p>
<p>After setting register <code>a</code> to <code>1</code>, if the program were to run to completion, <em>what value would be left in register <code>h</code>?</em></p>
</article>

</main>


In [19]:
def coprosser_II(program: str) -> int:
    registers = defaultdict(int)
    registers["a"] = 1

    regex = compile(r"^[\-+]?\d+$")

    instuctions = [
        [int(c) if match(regex, c) else c for c in line.split(" ")]
        for line in program.strip().splitlines()
    ]

    counter, count_mul = 0, 0
    while 0 <= counter < len(instuctions):
        match instuctions[counter]:
            case ["set", str(x), str(y)]:
                registers[x] = registers[y]
            case ["set", str(x), int(y)]:
                registers[x] = y
            case ["add", str(x), str(y)]:
                registers[x] += registers[y]
            case ["add", str(x), int(y)]:
                registers[x] += y
            case ["mul", str(x), str(y)]:
                count_mul += 1
                registers[x] *= registers[y]
            case ["mul", str(x), int(y)]:
                count_mul += 1
                registers[x] *= y
            case ["mod", str(x), str(y)]:
                registers[x] %= registers[y]
            case ["mod", str(x), int(y)]:
                registers[x] %= y
            case ["jnz", str(x), str(y)]:
                if registers[x] != 0:
                    counter += registers[y] - 1
            case ["jnz", str(x), int(y)]:
                if registers[x] != 0:
                    counter += y - 1
            case ["jnz", int(x), str(y)]:
                if x != 0:
                    counter += registers[y] - 1
            case ["jnz", int(x), int(y)]:
                if x != 0:
                    counter += y - 1
            case ["sub", str(x), str(y)]:
                registers[x] -= registers[y]
            case ["sub", str(x), int(y)]:
                registers[x] -= y
            case _:
                raise ValueError(f"Unknown instruction {instuctions[counter]}.")

        counter += 1

    return registers


def part2() -> int:
    """
    b = 79 * 100 + 100000  # 107900
    c = b + 17000
    h = 0

    for b in range(107900, c + 1, 17):
        f = 1
        # check for prime
        for d in range(2, b + 1):
            for e in range(2, b + 1):
                if d * e == b:
                    f = 0
        if f == 0:
            h += 1
    """
    h = 0
    for b in range(107900, 107900 + 17000 + 1, 17):
        for i in range(2, b):
            if b % i == 0:
                h += 1
                break
    return h


print("Part II:", part2())

Part II: 907


<link href="style.css" rel="stylesheet"></link><main>
<main>

<p>Your puzzle answer was <code>907</code>.</p><p class="day-success">Both parts of this puzzle are complete! They provide two gold stars: **</p>

</main>
